In [ ]:
import os

import numpy as np
from matplotlib import pyplot as plt
import h5py as h5

import specpy as sp

from pipeline2 import PipelineLevels, AcquisitionPipeline, DefaultNameHandler, HDF5RichData, HDF5DataStore
from pipeline2.taskgeneration import (AcquisitionTaskGenerator, JSONFileConfigLoader, DefaultFOVSettingsGenerator, 
                                      AlreadyImagedFOVFilter,PairedDefaultScanOffsetsSettingsGenerator,
                                      DefaultScanOffsetsSettingsGenerator, DefaultLocationKeeper,
                                      DefaultLocationRemover, DefaultStageOffsetsSettingsGenerator,
                                      SpiralOffsetGenerator, NewestSettingsSelector, NewestDataSelector,
                                      BoundingBoxLocationGrouper, DefaultScanModeSettingsGenerator,
                                      DefaultScanFieldSettingsGenerator)
from pipeline2.detection import (LegacySpotPairFinder, SimpleSingleChannelSpotDetector, SimpleNucleusMidplaneDetector,
                                 SimpleLegacyFocusHold, SimpleLocationRepeater, PairedLegacySpotPairFinder)
from pipeline2.imspector import ImspectorConnection, get_current_stage_coords
from pipeline2.stoppingcriteria import TimedStoppingCriterion

from pipeline2.taskgeneration import StitchedNewestDataSelector

In [ ]:
base_path = 'C:/Users//RESOLFT/Desktop/auto-pairs/'
sample_name = '20180706-tetraspeck'

save_path = os.path.join(base_path, sample_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

In [ ]:
# plotting params
%matplotlib inline
plt.rcParams['figure.figsize'] = [7, 7]

# imspector object
im = sp.Imspector()

# init levels and name handler
pll = PipelineLevels('overview', 'detail', 'sted')
name_handler = DefaultNameHandler(save_path, pll)

# abuse name_handler to get h5 filename
# calling with empty index gives just the random hash
h5file_name = name_handler.get_path([]).replace('.msr', '.h5')

# open resulting h5 file in context manager
with h5.File(h5file_name, mode='a') as h5fd:
    data_store = HDF5DataStore(h5fd, pll)

    # init pipeline
    pl = (AcquisitionPipeline('spot-pair-pipeline-1')
            .withImspectorConnection(ImspectorConnection(im).withVerbose())
            .withPipelineLevels(pll)
            .withNameHandler(name_handler)
            .withAddedStoppingCondition(TimedStoppingCriterion(1 * 60 * 60)))
    
    # attach h5 data store
    pl.withDataStorage(data_store)
        
    # overview task generator: same settings every time
    atg_overview = (AcquisitionTaskGenerator(pll.overview, 
                    DefaultLocationRemover(
                        JSONFileConfigLoader(['C:/Users/RESOLFT/Desktop/config_json/20180706_overview_60x_2ch.json'])
                    ),
                    DefaultStageOffsetsSettingsGenerator(SpiralOffsetGenerator().withStart(get_current_stage_coords(im))),
                    DefaultScanModeSettingsGenerator(['xyz'], True),
                    DefaultFOVSettingsGenerator([[5e-5, 5e-5, 1e-5]], [[3e-7, 3e-7, 4e-7]]),
                    DefaultStageOffsetsSettingsGenerator(SimpleLegacyFocusHold(NewestDataSelector(pl, pll.overview)).withVerbose(True))
                )
                .withDelay(.4))

    # init detector in overview
    detector = LegacySpotPairFinder(NewestDataSelector(pl, pll.overview), 2, [0.5, 0.5]).withPlotDetections(True).withVerbose()

    # detail task generation: settings from last overview + new FOV + grouped detections
    # optionally repeat measurement to check reproducibility
    atg_detail = AcquisitionTaskGenerator(pll.detail,
                             DefaultLocationRemover(
                                 JSONFileConfigLoader(['C:/Users/RESOLFT/Desktop/config_json/20180706_overview_60x_2ch.json'])
                             ),
                             DefaultLocationKeeper(NewestSettingsSelector(pl, pll.overview)),
                             DefaultScanModeSettingsGenerator(['xyz'], True),
                             DefaultFOVSettingsGenerator([[2e-6, 2e-6, 2e-6]], [[1e-7, 1e-7, 1e-7]]),
                             DefaultScanOffsetsSettingsGenerator(
                                 SimpleLocationRepeater(
                                     BoundingBoxLocationGrouper(
                                         detector, [2e-6, 2e-6, 2e-6]).withVerbose(False), n=1)
                             ))
    atg_detail.withDelay(.2)

    # overview callback: re-add overview
    pl.withCallbackAtLevel(atg_overview, pll.overview)
    # overview callback 2: detect & do detail
    pl.withCallbackAtLevel(atg_detail, pll.overview)


    # another detector in detail
    detector2 = PairedLegacySpotPairFinder(NewestDataSelector(pl, pll.detail), 3, [0.1, 0.1], medianRadius=10, channels=(1,0) ).withPlotDetections(True).withVerbose()

    # detail task generation: settings from last overview + new FOV
    # do every measurement 3 times to assess reproducibility
    atg_detail_more = AcquisitionTaskGenerator(pll.sted,
                             DefaultLocationRemover(
                                JSONFileConfigLoader([
                                    'C:/Users/RESOLFT/Desktop/config_json/20180706_2dsted_60x_ch2.json',
                                    'C:/Users/RESOLFT/Desktop/config_json/20180706_3dsted_60x_ch2.json',
                                    'C:/Users/RESOLFT/Desktop/config_json/20180706_3dsted_60x_ch2.json',
                                    'C:/Users/RESOLFT/Desktop/config_json/20180706_2dsted_60x_ch1.json',
                                    'C:/Users/RESOLFT/Desktop/config_json/20180706_3dsted_60x_ch1.json',
                                    'C:/Users/RESOLFT/Desktop/config_json/20180706_3dsted_60x_ch1.json'
                                ], None, False)
                             ),
                             DefaultLocationKeeper(NewestSettingsSelector(pl, pll.overview)),
                             DefaultScanModeSettingsGenerator([
                                                               'xy',
                                                               'xz',
                                                               'yz',
                                                               'xy',
                                                               'xz',
                                                               'yz'
                                                              ], False),
                             DefaultFOVSettingsGenerator([[5e-7, 5e-7, 5e-7]], [[10e-9, 10e-9, 10e-9]]),
                             PairedDefaultScanOffsetsSettingsGenerator(SimpleLocationRepeater(detector2, n=1), repeat_channel=3)
                       )
    atg_detail_more.withDelay(.2)
    
    # detail callback: detect & do more detail
    pl.withCallbackAtLevel(atg_detail_more, pll.detail)

    # call overview atg once to add first measurement to pipeline
    atg_overview(pl)

    # GO
    pl.run()
